In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/MyDrive/') # 구글 드라이브와 연동

%cd '/content/MyDrive/피클데이터가 있는 경로 복사'

import pickle

with open('quiz_data.pkl', 'rb') as f:
  data = pickle.load(f)

print(data)

In [ ]:
# Local
from google.colab import files
data = files.upload()

In [5]:
import time
def timer(func):
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    computation_time = end_time - start_time
    print(f"Executation time of {func.__name__}: {computation_time} seconds")
    return result
  return wrapper

In [1]:
import torch
import pickle

# data.pkl 파일을 열어 데이터 로드
with open('quiz_data.pkl', 'rb') as file:
    data = pickle.load(file)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_data = torch.tensor(data['x'], dtype=float, device=device)
y_data = torch.tensor(data['y'], dtype=float, device=device)

In [ ]:
print(len(data['x']))

100


In [17]:
# Parameter 3
import torch.optim as optim
import torch.nn as nn

@timer
def train_model(lr=0.2, epochs=1500):
  #Initialize parameters
  b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w2 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w3 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  parameters = [b, w, w2, w3]
  optimizer = optim.Adam(parameters, lr=lr)
  mse_loss = nn.MSELoss()

  for epoch in range(epochs):
    # Loss computation
    y_hat = b + w * x_train + w2 * (x_train ** 2) + w3 * (x_train ** 3)
    loss = mse_loss(y_hat, y_train)

    # Gradient computation desent
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  return b, w, w2, w3

In [7]:
def test_model(b, w, w2, w3, x_test, y_test) :
  y_hat = b + w * x_test + w2 * (x_test ** 2) + w3 * (x_test ** 3)
  error = (y_test - y_hat)
  loss = torch.mean(error ** 2)
  return loss

In [18]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

# K-fold
N = len(x_data)
idx = np.arange(N)
np.random.shuffle(idx) # 무작위로 섞기

K = 10
fold_indices = np.array_split(idx, K)
loss_list = []

for i in range(K):
  val_idx = fold_indices[i]
  train_idx = np.concatenate([fold_indices[j] for j in range(K) if j != i])
  # concatenate로 배열 연결

  x_train, y_train = x_data[train_idx], y_data[train_idx]
  x_val, y_val = x_data[val_idx], y_data[val_idx]

  # Train
  b_hat, w_hat, w2_hat, w3_hat = train_model()

  # Evaluate
  loss = test_model(b_hat, w_hat, w2_hat, w3_hat, x_val, y_val)
  loss_list.append(loss)
  print(f"{i+1}번째 MSE: {loss}")

# 평균 성능 출력
print("평균 MSE : ", sum(loss_list) / K)



Executation time of train_model: 1.65629243850708 seconds
1번째 MSE: 0.004314915305992531
Executation time of train_model: 1.6610283851623535 seconds
2번째 MSE: 0.0021011657753921137
Executation time of train_model: 1.6725521087646484 seconds
3번째 MSE: 0.03277666627824073
Executation time of train_model: 1.687002182006836 seconds
4번째 MSE: 0.02767754287127849
Executation time of train_model: 1.746171236038208 seconds
5번째 MSE: 0.009242954937735552
Executation time of train_model: 1.8680174350738525 seconds
6번째 MSE: 0.03211200915852543
Executation time of train_model: 1.9319489002227783 seconds
7번째 MSE: 0.024934467315586323
Executation time of train_model: 1.7189273834228516 seconds
8번째 MSE: 0.24993864174217242
Executation time of train_model: 1.6859955787658691 seconds
9번째 MSE: 0.012744818274962376
Executation time of train_model: 1.6858062744140625 seconds
10번째 MSE: 0.024014434457765202
평균 MSE :  tensor(0.0420, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


# 지하 창고

In [ ]:
# CPU to GPU

x_train = x_train.to(device)
y_train = y_train.to(device)
x_val = x_val.to(device)
y_val = y_val.to(device)

In [ ]:
b_hat, w_hat, w2_hat, w3_hat = train_model()
print("b_estimate: {}, w_estimate: {}, w2_estimate: {}, w3_estimate: {}".format(b_hat, w_hat, w2_hat, w3_hat))

epoch: 50, loss: 0.3031658029161025
epoch: 100, loss: 0.22477976020717955
epoch: 150, loss: 0.1822172544534384
epoch: 200, loss: 0.1642190601924547
epoch: 250, loss: 0.1566437185542531
epoch: 300, loss: 0.15210071609559936
epoch: 350, loss: 0.14807616281603111
epoch: 400, loss: 0.14397114364374436
epoch: 450, loss: 0.13970154055991577
epoch: 500, loss: 0.13528296487869512
epoch: 550, loss: 0.13074382654852182
epoch: 600, loss: 0.1261121435687858
epoch: 650, loss: 0.1214141742775282
epoch: 700, loss: 0.1166745406611567
epoch: 750, loss: 0.11191629892130167
epoch: 800, loss: 0.10716098292134142
epoch: 850, loss: 0.10242870689201547
epoch: 900, loss: 0.09773817584342709
epoch: 950, loss: 0.09310673379228646
epoch: 1000, loss: 0.08855026803218334
epoch: 1050, loss: 0.0840833694376007
epoch: 1100, loss: 0.07971925872098212
epoch: 1150, loss: 0.07546973689367178
epoch: 1200, loss: 0.07134538785081627
epoch: 1250, loss: 0.06735537651073269
epoch: 1300, loss: 0.06350760655977852
epoch: 1350, l

In [ ]:
iter_nums = 10
losses = []
for _ in range(iter_nums) :
  loss = test_model(b_hat, w_hat, w2_hat, w3_hat, x_val, y_val)
  losses.append(loss)
print("평균 MSE : ", sum(losses) / 10)

tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.0047070980072021484 seconds
tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.0019345283508300781 seconds
tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.0015308856964111328 seconds
tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.001638174057006836 seconds
tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.0015358924865722656 seconds
tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.001569509506225586 seconds
tensor(0.0291, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Executation time of test_model: 0.0014977455139160156 seconds
tensor(0.0291, device=

In [ ]:
import torch.optim as optim
import torch.nn as nn

@timer
def train_model_torch2(lr=0.1, epochs=1000):
  #Initialize parameters
  b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w2 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w3 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w4 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w5 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  parameters = [b, w, w2, w3, w4, w5]
  optimizer = optim.SGD(parameters, lr=lr)
  mse_loss = nn.MSELoss()

  for epoch in range(epochs):
    # Loss computation
    y_hat = b + w * x_train + w2 * (x_train ** 2) + w3 * (x_train ** 3) + w4 * (x_train ** 4) + w5 * (x_train ** 5)
    loss = mse_loss(y_hat, y_train)

    # Gradient computation desent
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 50 == 0:
      print("epoch: {}, loss: {}".format(epoch+1, loss))

  return b, w, w2, w3, w4, w5

In [ ]:
b_hat, w_hat, w2_hat, w3_hat, w4_hat, w5_hat = train_model_torch2()
print("b_estimate: {}, w_estimate: {}, w2_estimate: {}, w3_estimate: {}, w4_estimate: {}, w5_estimate: {}".format(b_hat, w_hat, w2_hat, w3_hat, w4_hat, w5_hat))

epoch: 50, loss: 0.2277517827308996
epoch: 100, loss: 0.22310602252734776
epoch: 150, loss: 0.21952271912624655
epoch: 200, loss: 0.21632518864325725
epoch: 250, loss: 0.2134091153754461
epoch: 300, loss: 0.21070156368062742
epoch: 350, loss: 0.20815040014902306
epoch: 400, loss: 0.2057183274165885
epoch: 450, loss: 0.2033786625429256
epoch: 500, loss: 0.2011122507651921
epoch: 550, loss: 0.19890535626847816
epoch: 600, loss: 0.19674807053138701
epoch: 650, loss: 0.19463324996799405
epoch: 700, loss: 0.19255570643116868
epoch: 750, loss: 0.19051164627839845
epoch: 800, loss: 0.1884982977479354
epoch: 850, loss: 0.18651356088580792
epoch: 900, loss: 0.1845559199520698
epoch: 950, loss: 0.1826241621399079
epoch: 1000, loss: 0.1807173612375285
Executation time of train_model_torch2: 1.1841440200805664 seconds
b_estimate: tensor([-0.7726], device='cuda:0', requires_grad=True), w_estimate: tensor([2.2976], device='cuda:0', requires_grad=True), w2_estimate: tensor([-0.7558], device='cuda:0',

In [ ]:
b_hat, w_hat, w2_hat, w3_hat, w4_hat = train_model_torch2()
print("b_estimate: {}, w_estimate: {}, w2_estimate: {}, w3_estimate: {}, w4_estimate: {}".format(b_hat, w_hat, w2_hat, w3_hat, w4_hat))

epoch: 50, loss: 0.3248355357498597
epoch: 100, loss: 0.2991291345501028
epoch: 150, loss: 0.2793031950382604
epoch: 200, loss: 0.26338916257300643
epoch: 250, loss: 0.2505637523229222
epoch: 300, loss: 0.24018237502013007
epoch: 350, loss: 0.23173530123726402
epoch: 400, loss: 0.22481942034985294
epoch: 450, loss: 0.21911592426497975
epoch: 500, loss: 0.21437268155526046
epoch: 550, loss: 0.2103902652220706
epoch: 600, loss: 0.20701097554099723
epoch: 650, loss: 0.2041100865666925
epoch: 700, loss: 0.20158894646591063
epoch: 750, loss: 0.1993695288137465
epoch: 800, loss: 0.19739010996889525
epoch: 850, loss: 0.19560188640575019
epoch: 900, loss: 0.19396623525079093
epoch: 950, loss: 0.19245262155777734
epoch: 1000, loss: 0.19103686527800756
Executation time of train_model_torch2: 0.9547667503356934 seconds
b_estimate: tensor([-0.7362], device='cuda:0', requires_grad=True), w_estimate: tensor([1.9130], device='cuda:0', requires_grad=True), w2_estimate: tensor([0.3111], device='cuda:0'